# 如何分发自定义回调事件

:::info 前置条件

本指南假设您已熟悉以下概念：

- [回调](/docs/concepts/callbacks)
- [自定义回调处理程序](/docs/how_to/custom_callbacks)
- [Astream 事件 API](/docs/concepts/streaming/#astream_events)，`astream_events` 方法将显示自定义回调事件。
:::

在某些情况下，您可能希望从 [Runnable](/docs/concepts/runnables) 内分发自定义回调事件，以便它可以在自定义回调处理程序或 [Astream 事件 API](/docs/concepts/streaming/#astream_events) 中显示出来。

例如，如果您有一个包含多个步骤的长时间运行工具，您可以在步骤之间分发自定义事件，并使用这些自定义事件来监控进度。您还可以将这些自定义事件显示给应用程序的最终用户，以向他们展示当前任务的进展情况。

要分发自定义事件，您需要为事件决定两个属性：`name` 和 `data`。

| 属性       | 类型 | 描述                                                                                              |
|------------|------|--------------------------------------------------------------------------------------------------|
| name       | str  | 事件的用户定义名称。                                                                             |
| data       | Any  | 与事件相关的数据。这可以是任何内容，但建议使其可 JSON 序列化。                                   |

:::important
* 分发自定义回调事件需要 `langchain-core>=0.2.15`。
* 自定义回调事件只能从现有的 `Runnable` 内分发。
* 如果使用 `astream_events`，您必须使用 `version='v2'` 才能看到自定义事件。
* 在 LangSmith 中发送或渲染自定义回调事件尚不支持。
:::

:::caution 兼容性
LangChain 无法自动将配置（包括 astream_events() 所需的回调）传播到子 Runnable，如果您在 python<=3.10 中运行异步代码。这是导致您可能无法看到自定义 Runnable 或工具发出的事件的常见原因。

如果您运行的是 python<=3.10，则需要在异步环境中手动将 `RunnableConfig` 对象传播到子 Runnable。有关如何手动传播配置的示例，请参见下面 `bar` RunnableLambda 的实现。

如果您运行的是 python>=3.11，则 `RunnableConfig` 将在异步环境中自动传播到子 Runnable。但是，如果您的代码可能在其他 Python 版本中运行，手动传播 `RunnableConfig` 仍然是一个好主意。
:::


In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain-core

## Astream 事件 API

使用 [Astream 事件 API](/docs/concepts/streaming/#astream_events) 是消费自定义事件的最有用方式。

我们可以使用异步的 `adispatch_custom_event` API 在异步环境中发出自定义事件。

:::important

要通过 astream 事件 API 查看自定义事件，您需要使用较新的 `v2` 版本的 `astream_events` API。
:::


In [3]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def foo(x: str) -> str:
    await adispatch_custom_event("event1", {"x": x})
    await adispatch_custom_event("event2", 5)
    return x


async for event in foo.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'foo', 'tags': [], 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'foo', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'foo', 'tags': [], 'metadata': {}, 'parent_ids': []}


在 python <= 3.10 中，您必须手动传播配置！

In [ ]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """一个展示如何手动传播配置的示例。

    如果您运行的是 python<=3.10，则必须这样做。
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async for event in bar.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'bar', 'tags': [], 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'bar', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'bar', 'tags': [], 'metadata': {}, 'parent_ids': []}


## 异步回调处理程序

您还可以通过异步回调处理程序消费分发的事件。

In [ ]:
from typing import Any, Dict, List, Optional
from uuid import UUID

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class AsyncCustomCallbackHandler(AsyncCallbackHandler):
    async def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"收到事件 {name}，数据：{data}，标签：{tags}，元数据：{metadata}，运行 ID：{run_id}"
        )


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """一个展示如何手动传播配置的示例。

    如果您运行的是 python<=3.10，则必须这样做。
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async_handler = AsyncCustomCallbackHandler()
await foo.ainvoke(1, {"callbacks": [async_handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: a62b84be-7afd-4829-9947-7165df1f37d9
Received event event2 with data: 5, with tags: ['foo', 'bar'], with metadata: {} and run_id: a62b84be-7afd-4829-9947-7165df1f37d9


1

## 同步回调处理程序

让我们看看如何在同步环境中使用 `dispatch_custom_event` 发出自定义事件。

您**必须**从现有的 `Runnable` 内调用 `dispatch_custom_event`。

In [ ]:
from typing import Any, Dict, List, Optional
from uuid: UUID

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.callbacks.manager import (
    dispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class CustomHandler(BaseCallbackHandler):
    def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"收到事件 {name}，数据：{data}，标签：{tags}，元数据：{metadata}，运行 ID：{run_id}"
        )


@RunnableLambda
def foo(x: int, config: RunnableConfig) -> int:
    dispatch_custom_event("event1", {"x": x})
    dispatch_custom_event("event2", {"x": x})
    return x


handler = CustomHandler()
foo.invoke(1, {"callbacks": [handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 27b5ce33-dc26-4b34-92dd-08a89cb22268
Received event event2 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 27b5ce33-dc26-4b34-92dd-08a89cb22268


1

## 下一步

您已经了解了如何发出自定义事件，您可以查看更深入的指南 [astream 事件](/docs/how_to/streaming/#using-stream-events)，这是利用自定义事件的最简单方法。